In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens
!pip uninstall jax jaxlib -y
!pip install jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingfa

In [1]:
from transformer_lens import HookedTransformer
import torch

import json
import time

from src.utils import tuple_str_to_tuple
from src.neuron_texts import get_neuron_max_acts

In [2]:
# Load jsons from ../experiment_data/1_next_token_neurons
# filename = "2023-10-11_19-25-40_gpt2-xl"
# filename = "2023-10-11_19-07-38_gpt2-small"
# filename = "2023-10-11_19-19-59_gpt2-medium"
filename = "2023-10-13_17-59-51_gpt2-large"


with open(f'./experiment_data/1_next_token_neurons/{filename}.json') as f:
    neurons_data = json.load(f)

In [3]:
# Parameters
parameters = neurons_data['parameters']
model_name = parameters['model_name']
neurons_list = [tuple_str_to_tuple(x) for x in neurons_data['neurons'].keys()]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
    device=device,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


In [5]:
# from datasets import load_dataset
# dataset = load_dataset("NeelNanda/pile-10k", split="train")
# dataset_text_list = [x['text'] for x in dataset]

import pickle

with open('./text_list_dict_test.pkl', 'rb') as f:
    dataset_text_dict = pickle.load(f)

dataset_text_list = [x['text'] for x in dataset_text_dict]

In [6]:
neuron_max_acts = get_neuron_max_acts(
    model=model,
    dataset_text_list=dataset_text_list,
    neurons_list=neurons_list,
    batch_size=1,
    device=device,
)

100%|██████████| 10000/10000 [50:57<00:00,  3.27it/s] 


In [7]:
output = {
    'parameters': parameters,
    'neuron_max_acts': {str(key): value for key, value in neuron_max_acts.items()},
    'prior_filename': filename,
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_mid_test.json"

with open(f'./experiment_data/2_max_activating_texts/{new_filename}', 'w') as f:
    json.dump(output, f)